# Demo SSL Handshake 

In [4]:
import os
import socket
import hmac
import hashlib
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.x509 import Certificate, load_pem_x509_certificate
from threading import Thread

# Fonction pour générer des nonces

In [5]:
def generate_nonce(length=32):
    return os.urandom(length)
    

# Fonction de génération de la paire de clés publique/privée avec RSA

In [6]:
def generate_rsa_key():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key
    

In [7]:
private_key, public_key = generate_rsa_key()

In [ ]:
# Signer le certificat avec la clé privée

def sign_certificate(private_key, public_key):
    public_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    signature = private_key.sign(
        public_bytes,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

certificate_signature = sign_certificate(private_key, public_key)

# Vérification du certificat

def verify_certificate(public_key, signature):
    try:
        public_key.verify(
            signature,
            public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("[+] Certificat vérifié avec succès")
    except Exception as e:
        print("[-] Certificat invalide:", e)

# Serveur : Accepte la connexion et effectue le handshake

def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(("localhost", 8080))
    server_socket.listen(1)
    print("[+] Serveur en attente de connexion...")

    conn, addr = server_socket.accept()
    print(f"[+] Connexion acceptée de {addr}")

    # Envoi du nonce serveur, de la clé publique et de la signature
    server_nonce = generate_nonce()
    conn.send(server_nonce)
    pem = public_key.public_bytes(encoding=serialization.Encoding.PEM,
                                  format=serialization.PublicFormat.SubjectPublicKeyInfo)
    conn.send(pem)
    conn.send(certificate_signature)

    # Réception de la clé pré-master chiffrée
    encrypted_key = conn.recv(256)
    pre_master_secret = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    print("[+] Pré-Master Secret reçu")

    # Génération de la clé de session
    session_key = hashlib.sha256(pre_master_secret + server_nonce).digest()
    print("[+] Clé de session générée")

    # Réception du message chiffré
    iv = conn.recv(12)
    ciphertext = conn.recv(1024)
    tag = conn.recv(16)

    decryptor = Cipher(algorithms.AES(session_key), modes.GCM(iv, tag)).decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    print(f"[+] Message reçu : {plaintext.decode()}")

    conn.close()
    print("[+] Connexion terminée")

# Client : Initie la connexion et effectue le handshake

def client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(("localhost", 8080))
    print("[+] Connecté au serveur")

    client_nonce = generate_nonce()

    # Réception du nonce serveur, de la clé publique et de la signature
    server_nonce = client_socket.recv(32)
    pem = client_socket.recv(1024)
    signature = client_socket.recv(256)
    server_public_key = serialization.load_pem_public_key(pem)

    # Vérification de la signature
    verify_certificate(server_public_key, signature)

    # Génération de la clé pré-master
    pre_master_secret = generate_nonce(48)
    encrypted_key = server_public_key.encrypt(
        pre_master_secret,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    client_socket.send(encrypted_key)
    print("[+] Pré-Master Secret envoyé")

    # Génération de la clé de session
    session_key = hashlib.sha256(pre_master_secret + server_nonce).digest()
    print("[+] Clé de session générée")

    # Chiffrement du message
    message = b"Hello SSL!"
    iv = generate_nonce(12)
    encryptor = Cipher(algorithms.AES(session_key), modes.GCM(iv)).encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()
    tag = encryptor.tag

    # Envoi du message chiffré
    client_socket.send(iv)
    client_socket.send(ciphertext)
    client_socket.send(tag)
    print("[+] Message chiffré envoyé")

    client_socket.close()

# Lancer le serveur et le client simultanément
server_thread = Thread(target=server)
server_thread.start()

import time
time.sleep(1)  # Petit délai pour que le serveur soit prêt

client_thread = Thread(target=client)
client_thread.start()

client_thread.join()
server_thread.join()

print("[+] Démonstration terminée")